In [131]:
import pygplates
import numpy as np
import matplotlib.pyplot as plt
#import pandas as pd
import glob

import splits_and_merges as snm
reload(snm)

%matplotlib inline

#######################################################
# Define Input files for Muller 2016 AREPS model
#######################################################


MODELDIR = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/'
RotFile_List = ['%s/Rotations/Matthews_etal_GPC_2016_410-0Ma_GK07.rot' % MODELDIR]
GPML_List = ['%s/DynamicPolygons/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies.gpmlz' % MODELDIR,\
             '%s/DynamicPolygons/Matthews_etal_GPC_2016_Paleozoic_PlateTopologies.gpmlz' % MODELDIR,\
             '%s/DynamicPolygons/Matthews_etal_GPC_2016_TopologyBuildingBlocks.gpmlz' % MODELDIR]


#MODELDIR = '/Users/Simon/Desktop/Global_Model_WD_Internal_Release_2016_v3_RM17/'
#RotFile_List = glob.glob('%s/*.rot' % MODELDIR)
#GPML_List = glob.glob('%s/*.gpml' % MODELDIR)


#####################################
rotation_model = pygplates.RotationModel(RotFile_List)
topology_features = pygplates.FeatureCollection()
for file in GPML_List:
    topology_feature = pygplates.FeatureCollection(file)
    topology_features.add(topology_feature)
#'''

pygplates.__version__


'2.0.0.12'

In [136]:
time_list = np.arange(50,20,-1)

for time in time_list:
    
    print '\n##############################\n\n'
        
    time_from = time+1
    time_to = time
    
    (appearing_plate_map,
     disappearing_plate_map) = snm.find_plate_change_for_time_step(topology_features,rotation_model,time_from,time_to)
                
    print '>>>>>>>> %s' % disappearing_plate_map
    
    new_plates_grouped_by_origin_plate = snm.match_splits_to_origin(appearing_plate_map)
        
    resolved_topologies = []
    shared_boundary_sections = []
    pygplates.resolve_topologies(topology_features, rotation_model, resolved_topologies, time_to, shared_boundary_sections)
    
    for plate_split in new_plates_grouped_by_origin_plate:
        
        if len(plate_split[1])!=2:
            print 'Only works for new splits with two plates'
        else:
            gca = snm.get_great_circle_along_plate_split(plate_split,
                                                     shared_boundary_sections)

            if gca is not None:
                print '>>>>>>>>>> Great Circle Normal is %s' % gca.get_great_circle_normal()

        


##############################


plates that persist between 51.00 Ma and 50.00 Ma are: 
 set([901, 902, 903, 904, 406, 409, 922, 801, 802, 804, 301, 304, 307, 308, 918, 701, 224, 201, 604, 608, 609, 738, 101, 102, 317, 501])

plates that appeared between 51.00 Ma and 50.00 Ma are: 
 set([2035])

plates that disappeared between 51.00 Ma and 50.00 Ma are: 
 set([])

Working on appearing plates
Centroid for plate 2035 mapped to plate 101 at delta time

Working on disappearing plates
>>>>>>>> []
Only works for new splits with two plates

##############################


plates that persist between 50.00 Ma and 49.00 Ma are: 
 set([918, 902, 903, 904, 406, 409, 922, 901, 801, 802, 804, 301, 304, 307, 308, 317, 224, 201, 604, 608, 609, 738, 101, 102, 701, 2035, 501])

plates that appeared between 50.00 Ma and 49.00 Ma are: 
 set([])

plates that disappeared between 50.00 Ma and 49.00 Ma are: 
 set([])

Working on appearing plates

Working on disappearing plates
>>>>>>>> []

###############

In [133]:
time_list = np.arange(50,0,-1)
verbose=False

for time in time_list:
    
    if verbose:
        print '\n##############################\n'
        
    time_from = time+1
    time_to = time
    
    (appearing_plate_map,
     disappearing_plate_map) = snm.find_plate_change_for_time_step(topology_features,rotation_model,time_from,time_to,verbose)
                
    new_plates_grouped_by_origin_plate = snm.match_splits_to_origin(appearing_plate_map)
        
    resolved_topologies_time_to = []
    shared_boundary_sections_time_to = []
    pygplates.resolve_topologies(topology_features, 
                                 rotation_model, 
                                 resolved_topologies_time_to, 
                                 time_to, 
                                 shared_boundary_sections_time_to)
    
    resolved_topologies_time_from = []
    pygplates.resolve_topologies(topology_features, 
                                 rotation_model, 
                                 resolved_topologies_time_from, 
                                 time_from)
    
    
    for plate_split in new_plates_grouped_by_origin_plate:
        
        #print plate_split
        
        area_after_split = 0
        for topology in resolved_topologies_time_to:
            for plate_id in plate_split[1]:
                if topology.get_feature().get_reconstruction_plate_id()==plate_id:
                    area_after_split += topology.get_resolved_geometry().get_area()
                if topology.get_feature().get_reconstruction_plate_id()==plate_split[0]:
                    print 'Splitting plate still exists.....'
                    area_after_split += topology.get_resolved_geometry().get_area()

        for topology in resolved_topologies_time_from:
            if topology.get_feature().get_reconstruction_plate_id()==plate_split[0]:
                area_before_split = topology.get_resolved_geometry().get_area()

        area_change_ratio = area_after_split/area_before_split
        print 'Time is %d, Plates are %s, Area Ratio %0.5f' % (time_to, plate_split, area_change_ratio)

        #gca = get_great_circle_along_plate_split(plate_split,
        #                                         shared_boundary_sections_time_to)
        #if gca is not None:
        #    print '>>>>>>>>>> Great Circle Normal is %s' % gca.get_great_circle_normal()

        

Splitting plate still exists.....
Time is 50, Plates are (101, [2035]), Area Ratio 1.00165
Splitting plate still exists.....
Time is 48, Plates are (801, [861]), Area Ratio 0.99711
Splitting plate still exists.....
Splitting plate still exists.....
Splitting plate still exists.....
Time is 47, Plates are (608, [645, 665, 926]), Area Ratio 2.48827
Splitting plate still exists.....
Time is 47, Plates are (609, [688]), Area Ratio 1.06554
Time is 45, Plates are (738, [735]), Area Ratio 0.97248
Splitting plate still exists.....
Time is 45, Plates are (604, [603]), Area Ratio 1.00763
Splitting plate still exists.....
Time is 44, Plates are (901, [855]), Area Ratio 0.99751
Splitting plate still exists.....
Time is 40, Plates are (801, [9923]), Area Ratio 1.00144
Splitting plate still exists.....
Splitting plate still exists.....
Time is 40, Plates are (802, [818, 819]), Area Ratio 1.98210
Splitting plate still exists.....
Time is 40, Plates are (604, [673]), Area Ratio 1.00723
Splitting plate

In [139]:
tmp = [(328, 301), (304, 301), (308, 701), (855, 839), (317, 301)]
print zip(*tmp)


[(328, 304, 308, 855, 317), (301, 301, 701, 839, 301)]


In [154]:
time_list = np.arange(50,0,-1)
verbose=False

plate_disappearance_time_lut = []

for time in time_list:
    
    if verbose:
        print '\n##############################\n'
        
    time_from = time+1
    time_to = time
    
    (appearing_plate_map,
     disappearing_plate_map) = snm.find_plate_change_for_time_step(topology_features,rotation_model,time_from,time_to,verbose)
        
    if len(zip(*disappearing_plate_map))>0:
        print list(zip(*disappearing_plate_map)[0])
        for plate_id in list(zip(*disappearing_plate_map)[0]):
            plate_id
            plate_disappearance_time_lut.append((plate_id,time))
    
print plate_disappearance_time_lut

for plate_disappearance_time in plate_disappearance_time_lut:
    if plate_disappearance_time[0]==522:
        print plate_disappearance_time[1]


[409]
[738, 926]
[501]
[307, 918, 735]
[855]
[663, 617, 604]
[9923]
[609]
[328, 304, 308, 855, 317]
[102]
[665]
[613, 838, 2035, 857]
[3201]
[804]
[835, 698, 861]
[689]
[688]
[306, 708, 522]
[853, 839]
[673]
[922]
[902, 971, 603, 604]
[615]
[819, 972]
[564]
[904, 969]
[(409, 46), (738, 45), (926, 45), (501, 43), (307, 40), (918, 40), (735, 40), (855, 38), (663, 37), (617, 37), (604, 37), (9923, 35), (609, 34), (328, 33), (304, 33), (308, 33), (855, 33), (317, 33), (102, 32), (665, 31), (613, 30), (838, 30), (2035, 30), (857, 30), (3201, 28), (804, 26), (835, 25), (698, 25), (861, 25), (689, 24), (688, 23), (306, 20), (708, 20), (522, 20), (853, 18), (839, 18), (673, 14), (922, 12), (902, 10), (971, 10), (603, 10), (604, 10), (615, 7), (819, 5), (972, 5), (564, 4), (904, 3), (969, 3)]
20
